In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 LangChain 在大型文件中進行問答 🦜🔗

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 查看 GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者| [Lavi Nigam](https://github.com/lavinigam-gcp) |


## 概述

這個筆記本展示如何使用 Vertex AI PaLM API，建構一個使用 LangChain 的問題解答 (Q&A) 系統，以從大型文件提取資訊。

在大型文件中建構問答系統的挑戰在於大型語言模型 (簡稱 LLM)，其權杖限制了可以使用多少內容。

有幾種方法可以提供文本。這些方法可以使用相似性搜尋或不使用。還有不同的方法可以將內容傳遞給 LLM。這個筆記本包含下列方法或鏈：

- **灌入** : 將整個文件內容作為內容。這是最簡單的方法，但對於大型文件可能會很沒效率。

- **Map-Reduce** : 將文件分成較小的區塊，並平行處理。這比灌入更有效率，但實作上會比較複雜。

- **調整** : 對一小部分執行初始提示，生成輸出，並針對每個後續文件，根據輸出和新文件調整輸出。這比 Map-Reduce 更準確，不過效率較低。

這個筆記本也展示使用向量相似性搜尋的「**Map-Reduce 與類似性搜尋** 」，其中你可以建立較小區塊的嵌入，並使用向量相似性搜尋找到相關內容。這是最有效率的方法，不過實作也可能最複雜。

瞭解更多關於 [LangChain](https://python.langchain.com/en/latest/use_cases/question_answering.html) 和 [Vertex Gen AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)


### 目標

在此教學課程中，你將學習如何執行以下操作：

- 擷取文件，其中包括下載文件。
- 使用 LangChain `PyPDFLoader` 從 PDF 萃取文字。
- 選取語境，用於辨識文件相關部分，以回答問題。
- 設計問題回答提示
- 針對處理大型語境，使用具備或不具備嵌入式的鏈


### 費用

本教學指南使用 Google Cloud 的計費元件：

* Vertex AI 生成式 AI Studio

了解 [Vertex AI 價格](https://cloud.google.com/vertex-ai/pricing)，
並使用 [價格計算器](https://cloud.google.com/products/calculator/)
根據預計使用情況產生費用估算值。


## 開始使用


### 安裝 Vertex AI SDK、其他套件及其相依性

安裝執行此雲端筆記本所需的下列套件。


In [ ]:
# Base system dependencies
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

# required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install poppler-utils python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
# Install Vertex AI LLM SDK, langchain and dependencies
! pip install google-cloud-aiplatform langchain==0.0.323 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client transformers==4.33.1 pypdf faiss-cpu config --user

### 僅 Colab：取消以下Cell註解，重新啟動核心。


***僅限 Colab** *：執行下列Cell重新啟動核心，或使用按鈕重新啟動核心。你的 Vertex AI Workbench 可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證你的筆記本環境

- 如果你使用 **Colab** 來執行這個筆記本，請執行以下單元格並繼續。
- 如果你使用的是 **Vertex AI Workbench** ，在此處查看設定說明 [link](https://github.com/doggy8088/generative-ai/tree/main/setup-env).


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

- 如果你是在本機開發環境中執行這個筆記本：
  - 安裝 [Google Cloud SDK](https://cloud.google.com/sdk)。
  - 取得驗證憑證。透過執行以下指令並遵循 oauth2 流程 (在此處進一步了解此指令 [here](https://cloud.google.com/sdk/gcloud/reference/beta/auth/application-default/login))，來建立本地憑證：

    ```bash
    gcloud auth application-default login
    ```


### 匯入函式庫


**僅限 Colab：** 執行下列Cell以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此操作。


In [ ]:
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

warnings.filterwarnings("ignore")
# restart python kernel if issues with langchain import.

### 匯入模型

你分別載入預先訓練好的文本和封裝產生模型，稱為 `text-bison@001` 和 `textembedding-gecko@001`


In [ ]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

## 使用大型文件進行問答

大型語言模型 (LLM) 是一款強大的工具，可用於回答與大型文件資料庫範圍廣泛相關的問題。但是，使用 LLM 進行問答會遇到一些挑戰。挑戰之一與 LLM 模型具有的有限知識有關，特別是在文件為特定內容時。

解決此限制的方法之一是使用檢索增強生成來提供有關文件的詳細資訊。檢索增強生成是一種利用 LLM 來回答其未接受過訓練的文件問題的技術。基本原理是先從語料庫中檢索任何相關文件，稱為「語境」，再將這些文件與原始問題一起傳遞給 LLM。然後，LLM 將根據已檢索文件的資訊產生回應。


### 匯入文件

要開始之前，你必須下載下列幾個檔案，這些檔案是下述摘要任務所需。


In [ ]:
data_folder = p.cwd() / "data"
p(data_folder).mkdir(parents=True, exist_ok=True)

pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = str(p(data_folder, pdf_url.split("/")[-1]))

urllib.request.urlretrieve(pdf_url, pdf_file)

### 從 PDF 中擷取文字

使用 `PdfReader` 從掃描文件擷取文字。


In [ ]:
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

### 提示設計

在問答系統中，你定義一個問題和相關提示。

問題只是一個字串，表示應用程式將被要求回答的問題。在此案例中，問題是 ```"什麼是實驗？"```

提示是一個字串，包含應用程式將用於產生對問題的回答之情境。在此案例中，提示是

```
使用提供的內容盡可能精確地回答問題。
如果答案未包含在內容中，請說「答案在內容中不可用」 \\n\\n

語境：\\n {context}？\\n
問題：\\n {question} \\n
答案：
```


In [ ]:
question = "What is Experimentation?"
prompt_template = """Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### 不帶相似性搜尋的問答

關於提供上下文，你可以提供它，或你可以使用你正在尋找答案的文字部分。

在此範例中，你選出前八頁做為你問答系統的上下文。


#### 背景選擇


In [ ]:
context = "\n".join(str(p.page_content) for p in pages[:7])
print("The total words in the context: ", len(context))

#### 問與答方法或鏈條


##### 方法 1：塞入

`塞入` 是一種將大型語言模型 (LLM) 應用到問答中的簡單方法。其涉及在提示中提供所有相關資料為語境，提供給大型語言模型。

在 LangChain 中，你可以將 `StuffDocumentsChain` 用作 `load_qa_chain` 方法的一部分。你需要的部分是將 `chain_type` 設定為鏈的 `stuff`。


In [ ]:
stuff_chain = load_qa_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

在你初始化  `load_qa_chain` 鏈後，你可以根據輸入文件回答問題。


In [ ]:
stuff_answer = stuff_chain(
    {"input_documents": pages[7:10], "question": question}, return_only_outputs=True
)

In [ ]:
pprint(stuff_answer)

「填充」方法的優點是只需呼叫 LLM 一次，但受限於 LLM 的脈絡長度，對於大量的資料而言並不可行。

以下是當脈絡達到 LLM 的限制時出現的異常情況。


In [ ]:
try:
    print(
        stuff_chain(
            {"input_documents": pages[7:], "question": question},
            return_only_outputs=True,
        )
    )
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

################### 方法 2：MapReduce

透過，你可以克服文字上限。它包含將文件分為多個段落、為每個段落執行初始提示，然後使用不同的提示合併初始提示結果。

在 LangChain 中，你可以將用於`load_qa_chain`方法的一部分，且鏈的`chain_type`為`map_reduce`。

`load_qa_chain`使用`map_reduce`作為`chain_type`需要兩個提示：問題和合併提示。

問題提示用於要求 LLM 根據提拱的內容回答問題。在此情況下，`question_prompt`為

```
根據提供的內容盡可能精確地回答問題。 \n\n
內容：\n {context} \n
問題：\n {question} \n
答案：
```

合併提示物件用於合併提取的內容和問題，以建立最終答案。在此情況下，`combine_prompt`為

```
根據提取的內容和問題建立最終答案。
如果答案不包含於內容中，請表示「答案未出現在內容中」。 \n\n
摘要：\n {summaries}?\n
問題：\n {question} \n
答案：
```


In [ ]:
question_prompt_template = """
                    Answer the question as precise as possible using the provided context. \n\n
                    Context: \n {context} \n
                    Question: \n {question} \n
                    Answer:
                    """
question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

# summaries is required. a bit confusing.
combine_prompt_template = """Given the extracted content and the question, create a final answer.
If the answer is not contained in the context, say "answer not available in context. \n\n
Summaries: \n {summaries}?\n
Question: \n {question} \n
Answer:
"""
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)

定義預期的提示符後，初始化一個 `load_qa_chain` 鏈。


In [ ]:
map_reduce_chain = load_qa_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    return_intermediate_steps=True,
    question_prompt=question_prompt,
    combine_prompt=combine_prompt,
)

你根據輸入文件回答你的問題。請注意你如何傳遞整個文件數據庫。


In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages, "question": question})

你可以將答案儲存在 Pandas 資料框中以檢查 `MapReduce` 中間步驟與 LLM 的回答。


In [ ]:
final_mp_data = []

# for each document, extract metadata and intermediate steps of the MapReduce process
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mp_data.append(output)

In [ ]:
# create a dataframe from a dictionary
pdf_mp_answers = pd.DataFrame.from_dict(final_mp_data)
# sorting the dataframe by filename and page_number
pdf_mp_answers = pdf_mp_answers.sort_values(by=["file_name", "page_number"])
pdf_mp_answers.reset_index(inplace=True, drop=True)
pdf_mp_answers.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_mp_answers["chunks"].iloc[index])
print("\n\n [Answer]")
print(pdf_mp_answers["answer"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_answers["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_answers["file_name"].iloc[index])

In [ ]:
index = 5
print("[Context]")
print(pdf_mp_answers["chunks"].iloc[index])
print("\n\n [Answer]")
print(pdf_mp_answers["answer"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_answers["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_answers["file_name"].iloc[index])

**考量：** `MapReduce` 方法有優點，就是能擴充至比填塞方法更大量的資料，但會需要更多呼叫 LLM，而且在最後組合呼叫時可能會遺失部分資訊。


##### 方法 3：Refine

使用 `Refine` 方法嘗試克服 `MapReduce` 方法的 `資訊` 遺失。此方法包含對第一塊資料執行初始提示、產生輸出。對於其餘文件，將該輸出與下一個文件一同傳入，要求 LLM 根據新文件，對輸出進行優化。

在 LangChain 中，你可以使用 `MapReduceDocumentsChain` 作為 `load_qa_chain` 方法的一部分。你所需要做的是將 `refine` 設定為你的鏈的 `chain_type`。

將 `refine` 設定為 chain_type 的 `load_qa_chain` 需要兩個提示：優化提示和初始問題提示。

`優化提示` 用於產生一個提示，要求 LLM 根據提供的脈絡來優化既有答案。在此例中，`優化提示` 是：

```
原始問題為：\n {question} \n
提供的答案為：\n {existing_answer}\n
根據下列脈絡，對既有答案進行優化 (如果需要)：\n {context_str} \n
根據萃取的內容和問題，產生最終答案。
如果答案未包含在脈絡中，請回答「答案不在脈絡中」。 \n\n
```

`初始問題` 提示用於產生一個提示，要求 LLM 僅根據提供的脈絡來回答問題。在此例中，`初始問題提示` 是：

```
根據提供的脈絡盡可能精確回答問題。\n\n
脈絡：\n {context_str} \n
問題：\n {question} \n
答案：
```


In [ ]:
refine_prompt_template = """
    The original question is: \n {question} \n
    The provided answer is: \n {existing_answer}\n
    Refine the existing answer if needed with the following context: \n {context_str} \n
    Given the extracted content and the question, create a final answer.
    If the answer is not contained in the context, say "answer not available in context. \n\n
"""
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)


initial_question_prompt_template = """
    Answer the question as precise as possible using the provided context only. \n\n
    Context: \n {context_str} \n
    Question: \n {question} \n
    Answer:
"""

initial_question_prompt = PromptTemplate(
    input_variables=["context_str", "question"],
    template=initial_question_prompt_template,
)

定義預期的提示符後，初始化一個 `load_qa_chain` 鏈。


In [ ]:
refine_chain = load_qa_chain(
    vertex_llm_text,
    chain_type="refine",
    return_intermediate_steps=True,
    question_prompt=initial_question_prompt,
    refine_prompt=refine_prompt,
)

你根據輸入文件回答你的問題。請注意你如何傳遞整個文件數據庫。


In [ ]:
refine_outputs = refine_chain({"input_documents": pages, "question": question})

你可以在 Pandas dataframe 中儲存答案，以檢查「Refine」中間步驟和 LLM 答案。


In [ ]:
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_refine_data.append(output)

In [ ]:
pdf_refine_answers = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_answers = pdf_refine_answers.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_refine_answers.reset_index(inplace=True, drop=True)
pdf_refine_answers.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_refine_answers["chunks"].iloc[index])
print("\n\n [Answer]")
print(pdf_refine_answers["answer"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_answers["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_answers["file_name"].iloc[index])

In [ ]:
index = 5
print("[Context]")
print(pdf_refine_answers["chunks"].iloc[index])
print("\n\n [Answer]")
print(pdf_refine_answers["answer"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_answers["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_answers["file_name"].iloc[index])

**考量** : 到目前為止，你使用文件的部分或整個文件作為背景，來回答你的具體問題。這兩種情況都有幾個限制，包括背景不完整且查詢速度慢，大型文件情況特別明顯。

透過向量資料庫進行相似性搜尋是一種較新的方法，可以解決這些限制。


### 與相似度搜尋的問答

利用向量資料庫進行相似度搜尋時，會將每個內容片段表示為一個向量。這些向量會儲存在資料庫中。當使用者提出問題時，系統首先會計算問題與資料庫中各向量的相似度。相似度最高的那幾個向量將會被用於擷取與問題相關的內容。

這種方法具備數種優點，包括回應更符合使用者的問題。

在這個案例中，你使用開源的內建式嵌入資料庫「Chroma」來建立相似度搜尋索引。


#### 背景選擇


使用 `Chroma` 建立相似度搜尋索引。

`Chroma` 與像 `PyPDFLoader` 的文件載入器搭配使用。


In [ ]:
vector_index = Chroma.from_documents(pages, vertex_embeddings).as_retriever()

接著，使用原始問題擷取相關的內容


In [ ]:
docs = vector_index.get_relevant_documents(question)

#### MapReduce 方法

最後根據你從嵌入式資料庫中檢索到的內容以及輸入的問題回答你的問題。


In [ ]:
map_reduce_embeddings_outputs = map_reduce_chain(
    {"input_documents": docs, "question": question}
)

In [ ]:
print(map_reduce_embeddings_outputs["output_text"])

你可以將答案儲存於 Pandas 資料框中，以檢查 `MapReduce with similarity search` 的中間步驟及 LLMs 的答案。


In [ ]:
final_mpe_data = []
for doc, out in zip(
    map_reduce_embeddings_outputs["input_documents"],
    map_reduce_embeddings_outputs["intermediate_steps"],
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mpe_data.append(output)

In [ ]:
pdf_mpe_answers = pd.DataFrame.from_dict(final_mpe_data)
pdf_mpe_answers = pdf_mpe_answers.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mpe_answers.reset_index(inplace=True, drop=True)
pdf_mpe_answers.head()

你可以將答案儲存在 Pandas dataframe 中，以檢查「搭配相似度搜尋的 MapReduce」中間步驟和 LLM 答案。


In [ ]:
final_mpe_data = []
for doc, out in zip(
    map_reduce_embeddings_outputs["input_documents"],
    map_reduce_embeddings_outputs["intermediate_steps"],
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mpe_data.append(output)

In [ ]:
pdf_mpe_answers = pd.DataFrame.from_dict(final_mpe_data)
pdf_mpe_answers = pdf_mpe_answers.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mpe_answers.reset_index(inplace=True, drop=True)
pdf_mpe_answers.head()

## 結論

這個筆記本示範了如何使用 LangChain 與 Vertex AI PaLM API 建立一個問答 (QA) 系統，以從大型文件提取資訊。

在此案例中，你使用 Chroma (一個記憶體中開源嵌入式資料庫) 來建立相似性搜尋索引。不過 [LangChain](https://python.langchain.com/docs/integrations/vectorstores/matchingengine) 支援 Vertex AI Matching Engine，這是一個 Google Cloud 的大型低延遲向量資料庫。透過 Vertex AI Matching Engine，你有一個全受控的服務，它可以擴充，以滿足最嚴苛的應用程式需求。它在訓練和推論時提供高執行效能。而且，它有許多功能，包括支援多種類似性指標、批次推論和線上學習。這些功能對於需要執行複雜的匹配作業或需要能夠適應變更資料的應用程式來說非常重要。
